<a href="https://colab.research.google.com/github/SarahY89/Titanic-project-/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project on using tensorflow on titanic
1. Data Preparation
The purpose of the Titanic dataset is to predict whether the given passengers could be survived after Titinic hit the iceburg titanic, according to their personal information.

We usually use DataFrame from the pandas library to pre-process the structured data.

2. Model Definition
Usually there are three ways of modeling using APIs of Keras: sequential modeling using Sequential() function, arbitrary modeling using API functions, and customized modeling by inheriting base class Model.

Here we take the simplest way: sequential modeling using function Sequential().

3. Model Training
There are three usual ways for model training: use internal function fit, use internal function train_on_batch, and customized training loop. Here we introduce the simplist way: using internal function fit.

In [2]:
import warnings
warnings.filterwarnings('ignore',category=UserWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import plotly.express as px
import plotly.graph_objs as go
from tensorflow.keras import models,layers
from sklearn.model_selection import train_test_split


In [4]:
dftrain_raw = pd.read_csv('/train.csv')
dftest_raw = pd.read_csv('/test.csv')
dftrain_raw.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
%matplotlib inline
# Count the values of 'Survived' column
counts = dftrain_raw['Survived'].value_counts()

# Create the bar chart using Plotly Express
fig = px.bar(x=counts.index, y=counts.values,
             labels={'x': 'Survived', 'y': 'Counts'},
             title='Survived Counts',
             width=800, height=500)

# Show the plot
fig.show()

In [ ]:
# Create the histogram using Plotly Express
fig = px.histogram(dftrain_raw, x='Age', nbins=20, color_discrete_sequence=['purple'],
                   labels={'Age': 'Age', 'count': 'Frequency'},
                   title='Age Distribution',
                   width=800, height=500)

# Show the plot
fig.show()

In [8]:
# Create the traces for both 'Survived==0' and 'Survived==1'
trace_survived_0 = go.Histogram(x=dftrain_raw.query('Survived == 0')['Age'], opacity=0.7,
                                name='Survived==0', marker=dict(color='blue'))
trace_survived_1 = go.Histogram(x=dftrain_raw.query('Survived == 1')['Age'], opacity=0.7,
                                name='Survived==1', marker=dict(color='yellow'))

# Create the figure with the overlaid density plots
fig = go.Figure(data=[trace_survived_0, trace_survived_1])

# Update the layout to add axis labels and title
fig.update_layout(title='Age Density by Survival',
                  xaxis_title='Age', yaxis_title='Density')

# Show the plot
fig.show()

In [ ]:
def preprocessing(dfdata):
    dfresult= pd.DataFrame()

    #Pclass
    dfPclass = pd.get_dummies(dfdata['Pclass'])
    dfPclass.columns = ['Pclass_' +str(x) for x in dfPclass.columns ]
    dfresult = pd.concat([dfresult,dfPclass],axis = 1)

    #Sex
    dfSex = pd.get_dummies(dfdata['Sex'])
    dfresult = pd.concat([dfresult,dfSex],axis = 1)

    #Age
    dfresult['Age'] = dfdata['Age'].fillna(0)
    dfresult['Age_null'] = pd.isna(dfdata['Age']).astype('int32')

    #SibSp,Parch,Fare
    dfresult['SibSp'] = dfdata['SibSp']
    dfresult['Parch'] = dfdata['Parch']
    dfresult['Fare'] = dfdata['Fare']

    #Carbin
    dfresult['Cabin_null'] =  pd.isna(dfdata['Cabin']).astype('int32')

    #Embarked
    dfEmbarked = pd.get_dummies(dfdata['Embarked'],dummy_na=True)
    dfEmbarked.columns = ['Embarked_' + str(x) for x in dfEmbarked.columns]
    dfresult = pd.concat([dfresult,dfEmbarked],axis = 1)

    return dfresult
# Preprocess the data
x_train = preprocessing(dftrain_raw)
y_train = dftrain_raw['Survived'].values

# Split the data into training and testing sets
train_size = 0.8  # You can adjust the train_size as needed
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=train_size, random_state=42)

print("x_train.shape =", x_train.shape)

x_train.shape = (712, 15)


In [ ]:
tf.keras.backend.clear_session()

model = models.Sequential()
model.add(layers.Dense(20,activation = 'relu',input_shape=(15,)))
model.add(layers.Dense(10,activation = 'relu' ))
model.add(layers.Dense(1,activation = 'sigmoid' ))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                320       
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 541
Trainable params: 541
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Use binary cross entropy loss function for binary classification
model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['AUC'])

history = model.fit(x_train,y_train,
                    batch_size= 64,
                    epochs= 30,
                    validation_split=0.2 #Split part of the training data for validation
            )

Epoch 1/30
9/9 [==============================] - 2s 45ms/step - loss: 0.7250 - auc: 0.6267 - val_loss: 0.6826 - val_auc: 0.6705
Epoch 2/30
9/9 [==============================] - 0s 7ms/step - loss: 0.6794 - auc: 0.6487 - val_loss: 0.6453 - val_auc: 0.6818
Epoch 3/30
9/9 [==============================] - 0s 8ms/step - loss: 0.6504 - auc: 0.6585 - val_loss: 0.6306 - val_auc: 0.6906
Epoch 4/30
9/9 [==============================] - 0s 9ms/step - loss: 0.6310 - auc: 0.6696 - val_loss: 0.6150 - val_auc: 0.7022
Epoch 5/30
9/9 [==============================] - 0s 9ms/step - loss: 0.6194 - auc: 0.6817 - val_loss: 0.6059 - val_auc: 0.7329
Epoch 6/30
9/9 [==============================] - 0s 8ms/step - loss: 0.6095 - auc: 0.6972 - val_loss: 0.5957 - val_auc: 0.7618
Epoch 7/30
9/9 [==============================] - 0s 8ms/step - loss: 0.6020 - auc: 0.7135 - val_loss: 0.5784 - val_auc: 0.7671
Epoch 8/30
9/9 [==============================] - 0s 9ms/step - loss: 0.5986 - auc: 0.7258 - val_loss: 

In [ ]:
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)

    # Convert 'epochs' to a list
    epochs_list = list(epochs)

    # Create traces for train and validation metrics
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=epochs_list, y=train_metrics, mode='lines+markers',
                             line=dict(dash='dash'), name='train_'+metric))
    fig.add_trace(go.Scatter(x=epochs_list, y=val_metrics, mode='lines+markers',
                             line=dict(dash='solid'), name='val_'+metric))

    # Update the layout and add the legend
    fig.update_layout(title='Training and Validation ' + metric,
                      xaxis_title='Epochs', yaxis_title=metric,
                      legend_title_text='Metrics', legend=dict(font=dict(size=12)))

    # Show the plot
    fig.show()
plot_metric(history,"loss")

In [ ]:
plot_metric(history,"auc")

In [ ]:
model.evaluate(x = x_test,y = y_test)

6/6 [==============================] - 0s 3ms/step - loss: 0.4952 - auc: 0.8526


[0.49516531825065613, 0.8525739908218384]

In [ ]:
#Predict the possiblities
model.predict(x_test[0:10])
#model(tf.constant(x_test[0:10].values,dtype = tf.float32)) #Identical way

1/1 [==============================] - 0s 99ms/step


array([[0.19778849],
       [0.29592264],
       [0.1736111 ],
       [0.75695235],
       [0.54758966],
       [0.8146909 ],
       [0.49041712],
       [0.25570473],
       [0.4616572 ],
       [0.61591864]], dtype=float32)

In [ ]:
# Predict the classes
predictions = model.predict(x_test[0:10])
# Round off the predictions
np.round(predictions)

1/1 [==============================] - 0s 22ms/step


array([[0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.]], dtype=float32)

In [ ]:
# Saving model structure and parameters

model.save('./data/keras_model.h5')

del model  #Deleting current model

# Identical to the previous one
model = models.load_model('./data/keras_model.h5')
model.evaluate(x_test,y_test)

6/6 [==============================] - 0s 3ms/step - loss: 0.4952 - auc: 0.8526


[0.49516531825065613, 0.8525739908218384]